### Library

In [42]:
import os
from dotenv import load_dotenv

from binance.client import Client

import pandas as pd
from time import sleep
from datetime import datetime
import ta

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

### TA Calculation Functions

#### RSI

In [43]:
def rsi(df, period=14):
    rsi = ta.momentum.RSIIndicator(pd.Series(df), window=period).rsi()
    return rsi

#### EMA

In [44]:
def ema(df, period=200):
    ema = ta.trend.EMAIndicator(pd.Series(df), period).ema_indicator()
    return ema

#### MACD

In [45]:
def macd(df):
    macd = ta.trend.MACD(pd.Series(df)).macd()
    return macd

#### Bollinger Bands

In [46]:
def signal_h(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_hband()
def signal_l(df):
    return ta.volatility.BollingerBands(pd.Series(df)).bollinger_lband()

### Strategy Class

#### RSI

In [133]:
class str(Strategy):
    # Any variables you want:
    ema_period = 200
    rsi_period = 14
    tp = 0.03
    sl = 0.02
    def init(self):
        # Take close prices as actual price
        price = self.data.Close
        # Declare indicators you will use in the strategy:
        self.rsi = self.I(rsi, self.data.Close, self.rsi_period)
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        # self.macd = self.I(macd, self.data.Close)
        # self.ema = self.I(ema, self.data.Close, self.ema_period)
        # self.bol_h = self.I(signal_h, self.data.Close)
        # self.bol_l = self.I(signal_l, self.data.Close)

    def next(self):
        # price = float(self.data.Close[-1])
        # # Strategy example. Its simple RSI. Buy when RSI<30 and sell when RSI>70
        # if not self.position and self.rsi[-2] < 30:
        #     # size is % of the 'cash'
        #     self.buy(size=0.02, tp=(1+tp)*price, sl=(1-sl)*price)
        # if not self.position and self.rsi[-2] > 70:
        #     self.sell(size=0.02, tp=(1-tp)*price, sl=(1+sl)*price)
        if crossover(self.ma1, self.ma2) and self.rsi[-2] < 30:
            self.buy()
        elif crossover(self.ma2, self.ma1) and self.rsi[-2] > 70:
            self.sell()

#### Simple Moving Average

In [123]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

## Main

### Binance API

In [124]:
load_dotenv()
API_KEY    = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')

client = Client(API_KEY, API_SECRET)

### Parameters

In [125]:
# Take Profit and Stop Loss. 0.03 means 3%
tp = 0.03
sl = 0.02
# Timeframe (for example, '1m', '3m', '5m', '15m', '1h', '4h')
# timeframe = ''
# Interval in days:
# interval = 30

### Fetch Historical Data

In [142]:
symbol = 'BTCUSDT'
interval = '1h'
# interval = '1h'  # 1-hour interval


start_time = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_time   = int(datetime(2021,12,31,0,0).timestamp() * 1000)

# Fetch the OHLCV data
ohlcv_data = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time, end_str=end_time)

data = pd.DataFrame(ohlcv_data)
data = data.iloc[:, :6]

data.columns = ['index'] + GOOG.columns

# data.columns = ['index',
#                 'Open',
#                 'High',
#                 'Low',
#                 'Close',
#                 'Volume']
data['index'] = pd.to_datetime(data['index'], unit='ms')
data.set_index('index', inplace=True)
data = data.astype(float)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements

### Backtesting

#### Backtesting

In [134]:
# bt = Backtest(GOOG, SmaCross, margin=1/2, commission=0.0007, exclusive_orders=True)
# bt = Backtest(data, SmaCross, margin=1/2, commission=0.0007, exclusive_orders=True)
bt = Backtest(data, str,      margin=1/2, commission=0.0007, exclusive_orders=True)

/var/folders/ft/h3fjv72s4jdftdb2szbgyggr0000gp/T/ipykernel_20419/3323089293.py:3: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, str,      margin=1/2, commission=0.0007, exclusive_orders=True)


In [135]:
stats = bt.run()

#### Results

In [136]:
print(stats)

Start                     2019-12-31 16:00:00
End                       2021-12-30 16:00:00
Duration                    730 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              559.762268
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

#### Graph

In [137]:
bt.plot()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample(freq, label='right').agg(OHLCV_AGG).dropna()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:128: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  indicators = [_Indicator(i.df.resample(freq, label='right').mean()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/backtesting/_plotting.py:128: FutureWarning: 'H' is depr

Row(id='16537', ...)

## For Temp Ad Hoc Testing

In [141]:
print(GOOG.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


In [139]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17490 entries, 2019-12-31 16:00:00 to 2021-12-30 16:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    17490 non-null  float64
 1   High    17490 non-null  float64
 2   Low     17490 non-null  float64
 3   Close   17490 non-null  float64
 4   Volume  17490 non-null  float64
dtypes: float64(5)
memory usage: 819.8 KB
None


In [147]:
data = ['index'].append(GOOG.columns)

In [151]:
['index'].entend(GOOG.columns)

Index(['OpenOpen', 'HighHigh', 'LowLow', 'CloseClose', 'VolumeVolume'], dtype='object')

## Reference

Backtesting User Manual

https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html